In [10]:
from dbt import tracking
from dbt.adapters.factory import get_adapter
from dbt.tracking import User

from dbt_utils import get_sql, get_compile_task, get_sql_improved
tracking.active_user = User('')
tracking.active_user.do_not_track = True

#sql = get_sql(model='stats_per_feature_day', vars={'feature_table': 'basic_features'})
#sql2 = get_sql_improved(model='stats_per_feature_day', vars={'feature_table': 'basic_features'})
ctask = get_compile_task(model='stats_per_feature_day', vars={'feature_table': 'basic_features'})
ctask.config.target_path
#ctask.graph is None
ctask.manifest is None
#graph_queue = ctask.get_graph_queue()
#print(sql)

True

In [14]:
ctask.load_manifest()
ctask.compile_manifest()
manifest = ctask.manifest
#ctask.graph

# graph = ctask.graph.graph
# node_id = 'model.objectiv.stats_per_feature_day'
# z = manifest.nodes['model.objectiv.stats_per_feature_day']
# #graph.manifest.nodes
# #x= graph.pred[node_id]
adapter = get_adapter(ctask.config)
compiler = adapter.get_compiler()
# selector = ctask.get_node_selector()
# spec = ctask.get_selection_spec()

In [15]:
graph_queue = ctask.get_graph_queue()
#graph_queue.get_selected_nodes()
# [graph_queue.get().unique_id
# ,graph_queue.get().unique_id
# ,graph_queue.get().unique_id
# ,graph_queue.get().unique_id
# ,graph_queue.get().unique_id]

result = []
while not graph_queue.empty():
    node = graph_queue.get()
    result.append(node.unique_id)
#'model.objectiv.stats_per_feature_day'
result
y = compiler.compile_node(node=node, manifest=manifest, extra_context=None, write=False)
sql = y.compiled_sql

In [16]:
import psycopg2
import psycopg2.extras
import os
POSTGRES_USER =     os.environ.get('POSTGRES_USER',     'objectiv')
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD', '0bj3ctiv')
POSTGRES_DB =       os.environ.get('POSTGRES_DB',       'objectiv')
POSTGRES_HOSTNAME = os.environ.get('POSTGRES_HOSTNAME', 'localhost')
POSTGRES_PORT =     os.environ.get('POSTGRES_PORT',     '5432')

connection = psycopg2.connect(user=POSTGRES_USER,
                            password=POSTGRES_PASSWORD,
                            host=POSTGRES_HOSTNAME,
                            port=POSTGRES_PORT,
                            database=POSTGRES_DB)

# This file should be mapped to a volume in the docker container,
# so changes to this file from within the container can be persisted.

with connection:
    with connection.cursor(cursor_factory=psycopg2.extras.DictCursor) as cursor:
        cursor.execute(sql)
        rows = cursor.fetchall()


In [18]:
#rows

In [19]:
#sql